# Cargar y utilzar modelo de MLflow directamente en script de Python

## Cargar el modelo

In [20]:
import dagshub
import mlflow
import mlflow.pyfunc
import pandas as pd

# Configurar la conexión a DagsHub
dagshub.init(repo_owner='VesnaPivac', repo_name='ML_COVID19-Mexico', mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/VesnaPivac/ML_COVID19-Mexico.mlflow")

# URI del modelo registrado en MLflow
model_name = "LogisticRegressionModel"  # Nombre del modelo que se desee utilizar
model_version = 1                       # Versión del modelo que se desee utilizar
model_uri = f"models:/{model_name}/{model_version}"

# Cargar el modelo desde MLflow
model = mlflow.pyfunc.load_model(model_uri)

Initialized MLflow to track repo "VesnaPivac/ML_COVID19-Mexico"

Repository VesnaPivac/ML_COVID19-Mexico initialized!

In [27]:
# Convertir los datos a un DataFrame
input_data = pd.read_parquet("../../../data/processed/X_test.parquet")[:10]
output_data = pd.read_parquet("../../../data/processed/y_test.parquet")[:10]

# Inspeccionar datos de entrada
input_data.head()

,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,INTUBADO,NEUMONIA,...,UCI,AÑO_ACTUALIZACION,MES_ACTUALIZACION,DIA_ACTUALIZACION,AÑO_INGRESO,MES_INGRESO,DIA_INGRESO,AÑO_SINTOMAS,MES_SINTOMAS,DIA_SINTOMAS
0,1,4,14,1,14,14,39,1,-2,2,...,-2,2024,4,23,2023,4,16,2023,4,15
1,2,4,29,2,29,29,41,1,-2,2,...,-2,2024,4,23,2023,1,18,2023,1,17
2,2,4,1,2,1,1,1,1,-2,2,...,-2,2024,4,23,2023,9,20,2023,9,17
3,2,9,25,1,25,25,1,1,-2,2,...,-2,2024,4,23,2023,9,1,2023,8,27
4,1,4,22,1,22,22,14,1,-2,2,...,-2,2024,4,23,2023,9,8,2023,9,7


In [38]:
# Realizar la predicción
predictions = model.predict(input_data)

# Mostrar los resultados
print(f"Predicciones del modelo: {predictions}")
print(f"Valores reales: {[x[0] for x in output_data.values]}")

Predicciones del modelo: [2 2 2 2 2 2 2 2 2 2]
Valores reales: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


### Ejemplo "What if"

Le platiqué a un amigo sobre el proyecto que estabamos desarrollando para la materia de aprendizaje automático y su hipocondría le hizo pedirme utilizar el modelo para diagnosticarlo. Le expliqué que mas bien el modelo predice si una persona vivirá o morirá dependiendo de sus características demográficas, comorbilidades y algunos otros datos. No le importó y procedió a proveerme sus datos. El resultado fue el siguiente:

In [85]:
# Cargar los archivos Excel
catalogos_path = "../../../references/processed/Catalogos.xlsx"
catalogos = pd.ExcelFile(catalogos_path)

# Mostrar las hojas de los archivos para entender su estructura
catalogos.sheet_names

['Catálogo ORIGEN',
 'Catálogo SECTOR',
 'Catálogo SEXO',
 'Catálogo TIPO_PACIENTE',
 'Catálogo SI_NO',
 'Catálogo NACIONALIDAD',
 'Catálogo RESULTADO_LAB',
 'Catálogo RESULTADO_ANTIGENO',
 'Catálogo CLASIFICACION_FINAL',
 'Catálogo de ENTIDADES',
 'Catálogo MUNICIPIOS',
 'Catálogo PAISES']

In [87]:
# Convertir catálogo en diccionario de listas
catalogs = {
    'ORIGEN': catalogos.parse('Catálogo ORIGEN')['CLAVE'].tolist(),
    'SECTOR': catalogos.parse('Catálogo SECTOR')['CLAVE'].tolist(),
    'ENTIDAD_UM': catalogos.parse('Catálogo de ENTIDADES')['CLAVE_ENTIDAD'].tolist(),
    'SEXO': catalogos.parse('Catálogo SEXO')['CLAVE'].tolist(),
    'ENTIDAD_NAC': catalogos.parse('Catálogo de ENTIDADES')['CLAVE_ENTIDAD'].tolist(),
    'ENTIDAD_RES': catalogos.parse('Catálogo de ENTIDADES')['CLAVE_ENTIDAD'].tolist(),
    'MUNICIPIO_RES': catalogos.parse('Catálogo MUNICIPIOS')['CLAVE_MUNICIPIO'].tolist(),
    'TIPO_PACIENTE': catalogos.parse('Catálogo TIPO_PACIENTE')['CLAVE'].tolist(),
    'INTUBADO': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'NEUMONIA': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'NACIONALIDAD': catalogos.parse('Catálogo NACIONALIDAD')['CLAVE'].tolist(),
    'EMBARAZO': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'HABLA_LENGUA_INDIG': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'INDIGENA': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'DIABETES': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'EPOC': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'ASMA': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'INMUSUPR': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'HIPERTENSION': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'OTRA_COM': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'CARDIOVASCULAR': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'OBESIDAD': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'RENAL_CRONICA': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'TABAQUISMO': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'OTRO_CASO': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'TOMA_MUESTRA_LAB': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'RESULTADO_LAB': catalogos.parse('Catálogo RESULTADO_LAB')['CLAVE'].tolist(),
    'TOMA_MUESTRA_ANTIGENO': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'RESULTADO_ANTIGENO': catalogos.parse('Catálogo RESULTADO_ANTIGENO')['CLAVE'].tolist(),
    'CLASIFICACION_FINAL': catalogos.parse('Catálogo CLASIFICACION_FINAL')['CLAVE'].tolist(),
    'MIGRANTE': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'PAIS_NACIONALIDAD': catalogos.parse('Catálogo PAISES')['CLAVE_PAIS'].tolist(),
    'PAIS_ORIGEN': catalogos.parse('Catálogo PAISES')['CLAVE_PAIS'].tolist(),
    'UCI': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
}

In [88]:
import random
from datetime import datetime, timedelta

# Función para generar fecha aleatoria entre un rango de años dado
def random_date(year_start, year_end):
    start_date = datetime(year_start, 1, 1)
    end_date = datetime(year_end, 12, 31)
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    return random_date

In [89]:
# Generar fechas
update_date = datetime.today()
symptoms_date = random_date(2020, 2024)
ingreso_date = symptoms_date + timedelta(days=random.randint(1, 5))

In [90]:
# Generar el Dataframe de una fila con los datos de mi amigo
data_amigo = {
    'ORIGEN': random.choice(catalogs['ORIGEN']),
    'SECTOR': random.choice(catalogs['SECTOR']),
    'ENTIDAD_UM': random.choice(catalogs['ENTIDAD_UM']),
    'SEXO': random.choice(catalogs['SEXO']),
    'ENTIDAD_NAC': random.choice(catalogs['ENTIDAD_NAC']),
    'ENTIDAD_RES': random.choice(catalogs['ENTIDAD_RES']),
    'MUNICIPIO_RES': random.choice(catalogs['MUNICIPIO_RES']),
    'TIPO_PACIENTE': random.choice(catalogs['TIPO_PACIENTE']),
    'INTUBADO': random.choice(catalogs['INTUBADO']),
    'NEUMONIA': random.choice(catalogs['NEUMONIA']),
    'EDAD': random.randint(0, 120),
    'NACIONALIDAD': random.choice(catalogs['NACIONALIDAD']),
    'EMBARAZO': random.choice(catalogs['EMBARAZO']),
    'HABLA_LENGUA_INDIG': random.choice(catalogs['HABLA_LENGUA_INDIG']),
    'INDIGENA': random.choice(catalogs['INDIGENA']),
    'DIABETES': random.choice(catalogs['DIABETES']),
    'EPOC': random.choice(catalogs['EPOC']),
    'ASMA': random.choice(catalogs['ASMA']),
    'INMUSUPR': random.choice(catalogs['INMUSUPR']),
    'HIPERTENSION': random.choice(catalogs['HIPERTENSION']),
    'OTRA_COM': random.choice(catalogs['OTRA_COM']),
    'CARDIOVASCULAR': random.choice(catalogs['CARDIOVASCULAR']),
    'OBESIDAD': random.choice(catalogs['OBESIDAD']),
    'RENAL_CRONICA': random.choice(catalogs['RENAL_CRONICA']),
    'TABAQUISMO': random.choice(catalogs['TABAQUISMO']),
    'OTRO_CASO': random.choice(catalogs['OTRO_CASO']),
    'TOMA_MUESTRA_LAB': random.choice(catalogs['TOMA_MUESTRA_LAB']),
    'RESULTADO_LAB': random.choice(catalogs['RESULTADO_LAB']),
    'TOMA_MUESTRA_ANTIGENO': random.choice(catalogs['TOMA_MUESTRA_ANTIGENO']),
    'RESULTADO_ANTIGENO': random.choice(catalogs['RESULTADO_ANTIGENO']),
    'CLASIFICACION_FINAL': random.choice(catalogs['CLASIFICACION_FINAL']),
    'MIGRANTE': random.choice(catalogs['MIGRANTE']),
    'PAIS_NACIONALIDAD': random.choice(catalogs['PAIS_NACIONALIDAD']),
    'PAIS_ORIGEN': random.choice(catalogs['PAIS_ORIGEN']),
    'UCI': random.choice(catalogs['UCI']),
    'AÑO_ACTUALIZACION': update_date.year,
    'MES_ACTUALIZACION': update_date.month,
    'DIA_ACTUALIZACION': update_date.day,
    'AÑO_INGRESO': ingreso_date.year,
    'MES_INGRESO': ingreso_date.month,
    'DIA_INGRESO': ingreso_date.day,
    'AÑO_SINTOMAS': symptoms_date.year,
    'MES_SINTOMAS': symptoms_date.month,
    'DIA_SINTOMAS': symptoms_date.day
}

In [93]:
# Crear el DataFrame de una sola fila
df_amigo = pd.DataFrame([data_amigo])

# Inspeccionar datos
df_amigo

,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,INTUBADO,NEUMONIA,...,UCI,AÑO_ACTUALIZACION,MES_ACTUALIZACION,DIA_ACTUALIZACION,AÑO_INGRESO,MES_INGRESO,DIA_INGRESO,AÑO_SINTOMAS,MES_SINTOMAS,DIA_SINTOMAS
0,2,7,10,2,24,29,11,2,1,2,...,-1,2024,5,30,2021,3,11,2021,3,9


In [94]:
# Realizar la predicción
predictions = model.predict(df_amigo)

# Mostrar los resultados
print(f"Predicciones del modelo: {predictions}")

Predicciones del modelo: [2]


Por el bien de la hipocondría de mi amigo, el modelo predice que dadas sus características él no fallecería.